TODO: 
- fix problem with spawning vehicles with scenario runner
    - probably based on position of spawnpoint based on waypoint, e.g. when increasing Z-value helps sometimes


1. Package runterladen +  conda env python 3.8
2. carla library installieren: carla/Dist/CARLA_Shipping_f38eea6e-dirty/LinuxNoEditor/PythonAPI/carla/dist/carla-0.9.14-cp38-cp38-linux_x86_64.whl


3. Start Caral Server, .exe ausführen
4. Load map: python3 repos/carla/PythonAPI/util/config.py --map=Town04
5. Start manual Car: python3 repos/carla/PythonAPI/examples/manual_control.py
 

In [1]:
import traceback
import pygame
import carla
import random
import time
import math
import collections
from collections import Counter
import pandas as pd
import itertools
from copy import deepcopy
import warnings
import numpy as np
import random
import sys
import os

warnings.filterwarnings("ignore")
from utils import *

pygame 2.5.2 (SDL 2.28.2, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
actor_list = []

for actor in actor_list:
    actor.destroy()
actor_list = []

client = carla.Client("localhost", 2000)
world = client.get_world()

blueprint_library = world.get_blueprint_library()
world_map = world.get_map()
spawn_points = world.get_map().get_spawn_points()
print(world.get_actors())

[Actor(id=8267, type=sensor.camera.rgb), Actor(id=8266, type=sensor.other.imu), Actor(id=8261, type=static.trigger.friction), Actor(id=8264, type=sensor.other.lane_invasion), Actor(id=8265, type=sensor.other.gnss), Actor(id=8262, type=vehicle.tesla.cybertruck), Actor(id=8263, type=sensor.other.collision), Actor(id=8256, type=traffic.traffic_light), Actor(id=8255, type=traffic.traffic_light), Actor(id=8254, type=traffic.traffic_light), Actor(id=8253, type=traffic.traffic_light), Actor(id=8252, type=traffic.traffic_light), Actor(id=8251, type=traffic.traffic_light), Actor(id=8250, type=traffic.traffic_light), Actor(id=8249, type=traffic.traffic_light), Actor(id=8248, type=traffic.traffic_light), Actor(id=8247, type=traffic.traffic_light), Actor(id=8246, type=traffic.traffic_light), Actor(id=8245, type=traffic.traffic_light), Actor(id=8244, type=traffic.traffic_light), Actor(id=8243, type=traffic.traffic_light), Actor(id=8242, type=traffic.traffic_light), Actor(id=8241, type=traffic.traff

In [6]:
# Set actor ID to your vehicle 
ego_vehicle = world.get_actor(8262)
#second_vehicle = world.get_actor(153)
print(ego_vehicle.get_location())

Location(x=20.182957, y=-288.970215, z=-0.003931)


## New Loop 


In [7]:
# initialize sensor data variables
matrix, speed, acceleration, steering_angle, lateral_acceleration, current_yaw, dist_to_lane_center, speed_vehicle_ahead, curvature_degrees_per_meter = None, None, None, None, None, None, None, None, None

previous_speed = 0  # Store the previous speed value in m/s
previous_yaw = 0  # Store the previous yaw angle value
leading_vehicle = None  # Variable to store the leading vehicle reference
previous_time = time.time_ns()

df = initialize_dataframe()
df_errors = initialize_error_dataframe()

try:
    while True:
        print("----------------------------------")
            
        # update ego location and waypoint
        ego_location = ego_vehicle.get_location()
        ego_waypoint = world_map.get_waypoint(ego_location)
            
        try:
            # create car detection matrix and update parameters
            matrix, params_car_detection = get_car_detection_matrix(**params_car_detection, ego_vehicle=ego_vehicle, ego_waypoint=ego_waypoint, ego_location=ego_location, world=world)
        except Exception as e:
            print("ERROR: In get_car_detection_matrix:")
            print(e)
            # print("Traceback:") # For debugging
            # traceback.print_exc()
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            new_row = {"location": ego_location, "error": e, "fname": fname, "line": exc_tb.tb_lineno, "exc_type": exc_type}
            df_errors = df_errors._append(new_row, ignore_index=True)

        current_time = time.time_ns()

        try:
            # get other sensor data
            speed = get_speed(ego_vehicle)
            acceleration = get_acceleration(speed, previous_speed, current_time, previous_time)
            steering_angle = get_steering_angle(ego_vehicle) #TODO angle in degree or radians?
            lateral_acceleration, current_yaw = get_lateral_acceleration(ego_vehicle, speed, previous_yaw, current_time, previous_time)
            dist_to_lane_center = get_dist_to_lane_center(ego_vehicle, world) #in m #TODO needs to be tested with manual car
            speed_vehicle_ahead = get_speed_of_vehicle_ahead(ego_waypoint, world) 

            # Get and print the curvature at the ego vehicle's location in degrees per meter
            curvature_degrees_per_meter = get_curvature_at_location(ego_location, world) 

            # save temporary data
            previous_speed = speed  # Update the previous speed value
            previous_time = current_time
            previous_yaw = current_yaw  # Update the previous yaw angle value
        except Exception as e:
            print("ERROR: In getting sensor data:")
            print(e)
            # print("Traceback:") # For debugging
            # traceback.print_exc()
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            new_row = {"location": ego_location, "error": e, "fname": fname, "line": exc_tb.tb_lineno, "exc_type": exc_type}
            df_errors = df_errors._append(new_row, ignore_index=True)

        # print sensor data
        print("matrix:")
        if matrix:
            for key, value in matrix.items():
                print(value)
        else:
            print("-- None --")
        print("speed", speed)
        print("acceleration", acceleration)
        print("steering_angle", steering_angle)
        print("lateral_acceleration", lateral_acceleration)
        print("dist_to_lane_center", dist_to_lane_center)
        print("speed_vehicle_ahead", speed_vehicle_ahead)
        print("curvature_degrees_per_meter", curvature_degrees_per_meter)


        # save sensor data in dataframe
        row_data = get_row(matrix)
        row_data["speed"] = speed
        row_data["acceleration"] = acceleration
        row_data["steering_angle"] = steering_angle
        row_data["lateral_acceleration"] = lateral_acceleration
        row_data["dist_to_lane_center"] = dist_to_lane_center
        row_data["speed_vehicle_ahead"] = speed_vehicle_ahead
        row_data["curvature_degrees_per_meter"] = curvature_degrees_per_meter
        df = df._append(row_data, ignore_index=True)

        # clock.tick_busy_loop(60)
        time.sleep(0.3)
        world.tick()
except:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    new_row = {"location": ego_location, "error": e, "fname": fname, "line": exc_tb.tb_lineno, "exc_type": exc_type}
    df_errors = df_errors._append(new_row, ignore_index=True)
    
finally: 
    display(df_errors)
    display(df)



----------------------------------
Error in recognizing junction shape
matrix:
-- None --
speed 0.0
acceleration 0.0
steering_angle 0.0
lateral_acceleration -0.0
dist_to_lane_center 0.17351631489369312
speed_vehicle_ahead 0.0
curvature_degrees_per_meter 0.006452695480474082
----------------------------------
Error in recognizing junction shape
matrix:
-- None --
speed 0.0
acceleration 0.0
steering_angle 0.0
lateral_acceleration 0.0
dist_to_lane_center 0.17351631489369312
speed_vehicle_ahead 0.0
curvature_degrees_per_meter 0.006452695480474082
----------------------------------
Error in recognizing junction shape
matrix:
-- None --
speed 0.0
acceleration 0.0
steering_angle 0.0
lateral_acceleration 0.0
dist_to_lane_center 0.17351631489369312
speed_vehicle_ahead 0.0
curvature_degrees_per_meter 0.006452695480474082
----------------------------------
Error in recognizing junction shape
matrix:
-- None --
speed 0.0
acceleration 0.0
steering_angle 0.0
lateral_acceleration 0.0
dist_to_lane_cen

,location,error,fname,line,exc_type
0,"Location(x=362.649048, y=-168.143082, z=0.209193)",list index out of range,2017891797.py,22,<class 'IndexError'>
1,"Location(x=367.251984, y=-168.102112, z=0.193769)",list index out of range,2017891797.py,22,<class 'IndexError'>
2,"Location(x=371.442413, y=-168.122925, z=0.193444)",list index out of range,2017891797.py,22,<class 'IndexError'>
3,"Location(x=383.460175, y=-157.533051, z=-0.001...",list index out of range,2017891797.py,22,<class 'IndexError'>
4,"Location(x=383.071075, y=-155.328796, z=0.000010)",list index out of range,2017891797.py,22,<class 'IndexError'>
5,"Location(x=382.614136, y=-152.858932, z=-0.002...",list index out of range,2017891797.py,22,<class 'IndexError'>
6,"Location(x=382.124634, y=-150.003052, z=-0.004...",list index out of range,2017891797.py,22,<class 'IndexError'>
7,"Location(x=381.803864, y=-146.527191, z=-0.001...",list index out of range,2017891797.py,22,<class 'IndexError'>
8,"Location(x=382.098755, y=-142.805527, z=0.003150)",list index out of range,2017891797.py,22,<class 'IndexError'>
9,"Location(x=382.481598, y=-139.573242, z=-0.000...",list index out of range,2017891797.py,22,<class 'IndexError'>


,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_0,1_1,...,7_5,7_6,7_7,speed,acceleration,steering_angle,lateral_acceleration,dist_to_lane_center,speed_vehicle_ahead,curvature_degrees_per_meter
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.000000e+00,-0.0,0.173516,0.0,0.006453
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.000000e+00,0.0,0.173516,0.0,0.006453
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.000000e+00,0.0,0.173516,0.0,0.006453
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.000000e+00,0.0,0.173516,0.0,0.006453
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.000000e+00,0.0,0.173516,0.0,0.006453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,0.0,0.0,0.0,5.133008e+19,0.0,4.938451,NaN,0.000000
239,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,0.0,0.0,0.0,5.133008e+19,0.0,4.938451,NaN,0.000000
240,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,0.0,0.0,0.0,5.133008e+19,0.0,4.938451,NaN,0.000000
241,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,0.0,0.0,0.0,5.133008e+19,0.0,4.938451,NaN,0.000000


NameError: name 'e' is not defined

In [ ]:
df.to_csv("test.csv")